In [1]:
from pysat.formula import CNF
import sys
import os
import math

In [2]:
import pysat
pysat.__version__

'0.1.7.dev12'

In [3]:
test_sats_path = 'test sudokus'

In [4]:
def read_DIMACS(path):
    return CNF(from_file=path).clauses
    
    
rules = read_DIMACS(os.path.join(test_sats_path, 'sudoku-rules-9x9.txt'))
rules[:10]

[[111, 112, 113, 114, 115, 116, 117, 118, 119],
 [-111, -112],
 [-111, -113],
 [-111, -114],
 [-111, -115],
 [-111, -116],
 [-111, -117],
 [-111, -118],
 [-111, -119],
 [-112, -113]]

In [5]:
input_mapping = {'0':0,'1':1,'2':2,'3':3,'4':4,'5':5,'6':6,'7':7,'8':8,'9':9, 'A': 10, 'B': 11, "C": 12, 'D': 13, 'E': 14, 'F': 15, 'G': 16}

inverse_input_mapping = {v:k for k, v in input_mapping.items()}
inverse_input_mapping

{0: '0',
 1: '1',
 2: '2',
 3: '3',
 4: '4',
 5: '5',
 6: '6',
 7: '7',
 8: '8',
 9: '9',
 10: 'A',
 11: 'B',
 12: 'C',
 13: 'D',
 14: 'E',
 15: 'F',
 16: 'G'}

In [6]:
def line2DIMACS(sudoku):
    input_len = len(sudoku)
    table_size = int(math.pow(input_len, 0.5))
    clause_list = []
    for row in range(table_size):
        for column in range(table_size):
            current_line_index = column + table_size * row
            cell_value = sudoku[current_line_index]
            if cell_value != '.':
                clause_list.append(f"{inverse_input_mapping[row+1]}{inverse_input_mapping[column+1]}{cell_value} 0")
    return clause_list, table_size

def line2CNF(line):
    cnf_list, table_size = line2DIMACS(line)
    return CNF(from_string='\n'.join(cnf_list)).clauses, cnf_list, table_size

In [7]:
# with open(os.path.join(test_sats_path, '4x4.txt')) as f:
with open(os.path.join(test_sats_path, '1000 sudokus.txt')) as f:
    sudoku_list = f.readlines()

In [49]:
sudoku_setup_CNF, sudoku_setup_DIMACS, sudoku_size = line2CNF(sudoku_list[1])

In [50]:
from dpll import DPLL

In [59]:
clauses = rules + sudoku_setup_CNF
# clauses = [[1,2], [1], [-3], [2, 3]]

solver = DPLL(clauses, variable_selection_method='random', verbose=0)

In [60]:
# solver.variables_set

In [61]:
# solver.clauses

In [62]:
%%time

solution = solver.backtrack(solver.clauses, partial_assignment=[], split_literal=tuple())

CPU times: user 29.1 s, sys: 851 ms, total: 30 s
Wall time: 30.5 s


In [14]:
solution

(True,
 [('117', True),
  ('111', False),
  ('112', False),
  ('113', False),
  ('114', False),
  ('115', False),
  ('116', False),
  ('118', False),
  ('119', False),
  ('127', False),
  ('137', False),
  ('217', False),
  ('227', False),
  ('237', False),
  ('317', False),
  ('327', False),
  ('337', False),
  ('147', False),
  ('157', False),
  ('167', False),
  ('177', False),
  ('187', False),
  ('197', False),
  ('417', False),
  ('517', False),
  ('617', False),
  ('717', False),
  ('817', False),
  ('917', False),
  ('138', True),
  ('131', False),
  ('132', False),
  ('133', False),
  ('134', False),
  ('135', False),
  ('136', False),
  ('139', False),
  ('128', False),
  ('218', False),
  ('228', False),
  ('238', False),
  ('318', False),
  ('328', False),
  ('338', False),
  ('148', False),
  ('158', False),
  ('168', False),
  ('178', False),
  ('188', False),
  ('198', False),
  ('438', False),
  ('538', False),
  ('638', False),
  ('738', False),
  ('838', False),
  ('9

In [15]:
def get_digit(number, n):
    return int(number // 10**n % 10)

import numpy as np
matrix = np.zeros((sudoku_size, sudoku_size), dtype=int)
for clause in sudoku_setup_CNF:
    for item in clause:
        matrix[get_digit(item, 2)-1, get_digit(item, 1)-1] = get_digit(item, 0)

In [16]:
def matprint(mat, fmt="g"):
    col_maxes = [max([len(("{:"+fmt+"}").format(x)) for x in col]) for col in mat.T]
    for x in mat:
        for i, y in enumerate(x):
            print(("{:"+str(col_maxes[i])+fmt+"}").format(y), end="  ")
        print("")
        
matprint(matrix)

7  0  8  9  0  0  0  0  0  
0  0  6  0  0  8  0  4  0  
0  0  0  2  5  0  0  3  0  
0  0  0  0  0  0  1  0  0  
6  0  0  0  1  0  0  2  0  
0  7  0  0  0  5  0  6  0  
0  0  0  0  7  0  0  0  0  
9  2  0  0  0  3  0  0  0  
1  0  0  0  8  0  6  0  0  


In [17]:
found_assignments = sorted(solution[1], key=lambda x: x[0])

matrix = np.zeros((sudoku_size, sudoku_size), dtype=int)
for assignment in found_assignments:
    if assignment[1]:
        item = assignment[0]
        matrix[int(item[0])-1, int(item[1])-1] = int(item[2])
        
matprint(matrix)

7  3  8  9  4  1  2  5  6  
2  5  6  7  3  8  9  4  1  
4  1  9  2  5  6  7  3  8  
5  8  2  6  9  4  1  7  3  
6  9  4  3  1  7  8  2  5  
3  7  1  8  2  5  4  6  9  
8  6  5  4  7  9  3  1  2  
9  2  7  1  6  3  5  8  4  
1  4  3  5  8  2  6  9  7  


# Experiment

In [20]:
counter = {('a', True): 0.25, ('b', False): 0.25}

In [21]:
variable_counter = {}
for variable in set(['a', 'b']):
    variable_counter[variable] = [counter.get((variable, True), 0), counter.get((variable, False), 0)]

In [22]:
variable_counter

{'a': [0.25, 0], 'b': [0, 0.25]}

In [29]:
max(variable_counter, key=lambda k: sum(variable_counter[k]))

'a'

In [30]:
def maxes(a, key=None):
    if key is None:
        key = lambda x: x
    m, max_list = key(a[0]), []
    for s in a:
        k = key(s)
        if k > m:
            m, max_list = k, [s]
        elif k == m:
            max_list.append(s)
    return m, max_list

In [38]:
list(variable_counter.keys())[0]

'a'

In [33]:
maxes(variable_counter, key=lambda k: sum(variable_counter[k]))

TypeError: unhashable type: 'list'